# Imports

In [109]:
import  tensorflow                  as      tf
import  numpy                       as      np
import  os
from    sklearn.model_selection     import  train_test_split
from    tensorflow.keras.utils      import  to_categorical

from    tensorflow.keras.models     import  Sequential
from    tensorflow.keras.layers     import  LSTM, Dense
from    tensorflow.keras.callbacks  import  TensorBoard

## Constants

In [110]:
SAMPLES_PER_WORD = 60
FRAMES_PER_SAMPLE = 30

SAMPLES_PATH = "/Volumes/SSD/collection/"

In [111]:
#words = np.array(['hello', 'my', 'none', 'person'])
words = np.array(['none', 'person', 'hello'])
words_path = {
    'hello' : 59,
    'none' : 61,
    'person' : 63
    }

labels_map = {label:num for num, label in enumerate(words)}
labels_map

{'none': 0, 'person': 1, 'hello': 2}

In [112]:

samples, labels = [], []
for word in words:
    for sample in range(SAMPLES_PER_WORD):
        sample_data = []
        for frame in range(FRAMES_PER_SAMPLE):
            path        = os.path.join(SAMPLES_PATH, f'{words_path[word]}', f'{words_path[word]}_{sample}_{frame}.npy')
            frame_data  = np.load(path)
            sample_data.append(frame_data)
        samples.append(sample_data)
        labels.append(labels_map[word])

In [113]:
X = np.array(samples)
X.shape

(180, 30, 225)

In [114]:
y = to_categorical(labels).astype(int)

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Model

In [116]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model           = Sequential()

hands   = 63 + 63
pose    = 99

lstm_model_1    = LSTM(     64,     return_sequences = True,    activation = "relu", input_shape=(30, hands+pose) )
lstm_model_2    = LSTM(     128,    return_sequences = True,    activation = "relu" )
lstm_model_3    = LSTM(     64,     return_sequences = False,   activation = "relu" )

dense_model_1   = Dense(    64,                                 activation = "relu" )
dense_model_2   = Dense(    32,                                 activation = "relu" )
dense_model_3   = Dense(    words.shape[0],                     activation = "softmax" )

model.add( lstm_model_1 )
model.add( lstm_model_2 )
model.add( lstm_model_3 )

model.add( dense_model_1 )
model.add( dense_model_2 )
model.add( dense_model_3 )

In [117]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [118]:
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - categorical_accuracy: 0.3271 - loss: 1.0905
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.6675 - loss: 0.7025
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - categorical_accuracy: 0.8556 - loss: 0.4619
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - categorical_accuracy: 0.9411 - loss: 0.2984
Epoch 5/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.8858 - loss: 0.3477
Epoch 6/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - categorical_accuracy: 0.9032 - loss: 0.2653
Epoch 7/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.9025 - loss: 0.2798
Epoch 8/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - categorical_accuracy: 0.9474 - loss: 0.2052
Epoch 9/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.9472 - loss: 0.1740
Epoch 10/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - categorical_accuracy: 0.9545 - loss: 0.1949
Epoch 11/1000
6/6 ━━━━━━━━━━━

In [120]:
model.save('new.keras')